In [1]:
from rumour_milled.ml.models.gan import HeadlinesGenerator, HeadlinesDiscriminator
from rumour_milled.ml.load import load_headlines
from rumour_milled.ml.preprocess import tokenise_and_vectorise
import torch

c:\Users\nedst\Documents\My Files\Projects\rumour-milled\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# GeForce Game Ready Driver 577.0
# GTX 1660
# Turing 7.5
# CUDA SDK 10.0-10.2

In [2]:
headlines, labels = load_headlines()
real_headlines = [
    headline
    for headline, label in zip(headlines, labels)
    if label == 1
]
real_headlines_subset = real_headlines[:512]

In [7]:
X = tokenise_and_vectorise(real_headlines_subset, batch_size=128)

Vectorising 128/512
Vectorising 256/512
Vectorising 384/512
Vectorising 512/512


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from rumour_milled.models import SimpleHeadlineClassifier

headlines, labels = load_headlines()
X = tokenise_and_vectorise(headlines, batch_size=512)
y = torch.tensor(labels, dtype=torch.float).unsqueeze(1)
dataset = TensorDataset(X, y)
data_loader = DataLoader(dataset, batch_size=10, shuffle=True)
model = SimpleHeadlineClassifier(768, 256, 1).to(device)
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy for [0,1] output
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Training loop
for epoch in range(1000):
    total_loss = 0
    for batch_X, batch_y in data_loader:
        optimizer.zero_grad()
        logits = model(batch_X.to(device))
        loss = criterion(logits, batch_y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [ ]:
with torch.no_grad():
    preds = model(tokenise_and_vectorise("Earth shattering headline"))
    print("Predictions:", preds.round())